In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('/content/drive/MyDrive/NLP/fake_or_real_news (1) (3)(4).csv')

In [3]:
data.head()

Unnamed: 0                                              title  \
0        8476                       You Can Smell Hillary’s Fear   
1       10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2        3608        Kerry to go to Paris in gesture of sympathy   
3       10142  Bernie supporters on Twitter erupt in anger ag...   
4         875   The Battle of New York: Why This Primary Matters   

                                                text label  
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4  It's primary day in New York and front-runners...  REAL

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data.shape

(6335, 4)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [7]:
data.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [8]:
data['label'].value_counts()

label
REAL    3171
FAKE    3164
Name: count, dtype: int64

In [9]:
data = data.drop(['Unnamed: 0','title'],axis = 1)

In [10]:
data.head()

text label
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE
2  U.S. Secretary of State John F. Kerry said Mon...  REAL
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE
4  It's primary day in New York and front-runners...  REAL

In [11]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [12]:
stem_port = PorterStemmer()

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [stem_port.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [15]:
data['text'] = data['text'].apply(stemming)

In [16]:
data.head()

text label
0  daniel greenfield shillman journal fellow free...  FAKE
1  googl pinterest digg linkedin reddit stumbleup...  FAKE
2  u secretari state john f kerri said monday sto...  REAL
3  kayde king kaydeek novemb lesson tonight dem l...  FAKE
4  primari day new york front runner hillari clin...  REAL

In [17]:
x = data['text'].values
y = data['label'].values

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x = vectorizer.transform(x)

In [21]:
print(x)

  (0, 43266)	0.0305089294798837
  (0, 43143)	0.018330936254574128
  (0, 42836)	0.03723430652002139
  (0, 42832)	0.024123281735309754
  (0, 42805)	0.03342157965058952
  (0, 42772)	0.02373158199733929
  (0, 42770)	0.008769006159949829
  (0, 42704)	0.01586413299776395
  (0, 42645)	0.016735394976427433
  (0, 42558)	0.028489938333701406
  (0, 42557)	0.01687960148746075
  (0, 42520)	0.0383844928542425
  (0, 42478)	0.04282388601939193
  (0, 42381)	0.034991614640544406
  (0, 42326)	0.03598071033413201
  (0, 42312)	0.01846435959961643
  (0, 42233)	0.013971188044141832
  (0, 42202)	0.06213972123547004
  (0, 42097)	0.011535614274685978
  (0, 42066)	0.027174758879218847
  (0, 41990)	0.028859093884229654
  (0, 41929)	0.032891625478617195
  (0, 41881)	0.01652269636508778
  (0, 41833)	0.053741305697756876
  (0, 41801)	0.07118481130240924
  :	:
  (6334, 3905)	0.02940423648335081
  (6334, 3904)	0.027151902202961768
  (6334, 3727)	0.03642146927330242
  (6334, 3720)	0.02107984191394252
  (6334, 3444)	0.0

In [22]:
x.shape


(6335, 43733)

In [50]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,stratify = y,random_state = 2)

MODEL BUILDING

In [51]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
y_pred

array(['REAL', 'FAKE', 'REAL', ..., 'FAKE', 'REAL', 'FAKE'], dtype=object)

In [52]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_pred,y_test)
accuracy

0.9221462388216728

In [65]:
news = 'Electric Floors Could Generate Renewable Energy From Walking'

In [66]:
news = stemming(news)

In [67]:
import numpy as np

In [68]:
model.predict(vectorizer.transform(np.array([news])))

array(['FAKE'], dtype=object)